In [24]:
from datetime import date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import Markdown as md
import plotly.io as pio

In [25]:
#Load covid 19 datset
df= pd.read_csv("owid-covid-data.csv")

In [26]:
df.shape

(152463, 67)

EDA

In [27]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [28]:
df.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152463 entries, 0 to 152462
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    152463 non-null  object 
 1   continent                                   143274 non-null  object 
 2   location                                    152463 non-null  object 
 3   date                                        152463 non-null  object 
 4   total_cases                                 149838 non-null  float64
 5   new_cases                                   149801 non-null  float64
 6   new_cases_smoothed                          148650 non-null  float64
 7   total_deaths                                132634 non-null  float64
 8   new_deaths                                  132822 non-null  float64
 9   new_deaths_smoothed                         132690 non-null  float64
 

Lot of data with NaN. And all data is not relevant for this EDA. For example, of the 152463 rows of data, columns like excess_mortality, excess_mortality_cumulative has data in only 5234 row (~3.4%). This is very little useable data in the context of all other available data.

We will create a subset of the df later. For now, lets delete the columns with data in <33% of the rows.

In [22]:
df_temp = df[[column for column in df if df[column].count() / len(df) >= .33]]
print("List of columns which has less data and can be dropped: \n", end="")
for c in df.columns:
    if c not in df_temp.columns:
        print(c, end=", ")

List of columns which has less data and can be dropped: 
icu_patients, icu_patients_per_million, hosp_patients, hosp_patients_per_million, weekly_icu_admissions, weekly_icu_admissions_per_million, weekly_hosp_admissions, weekly_hosp_admissions_per_million, total_vaccinations, people_vaccinated, people_fully_vaccinated, total_boosters, new_vaccinations, total_vaccinations_per_hundred, people_vaccinated_per_hundred, people_fully_vaccinated_per_hundred, total_boosters_per_hundred, excess_mortality_cumulative_absolute, excess_mortality_cumulative, excess_mortality, excess_mortality_cumulative_per_million, 

Columns which have all data.

In [30]:
df_temp = df[[column for column in df if df[column].count() / len(df) == 1.0]]
print("List of columns which have all data: \n", end="")
for c in df.columns:
    if c in df_temp.columns:
        print(c, end=", ")

List of columns which have all data: 
iso_code, location, date, 

In [32]:
#Splitting date into month and year, and creating a new cloumns for month and date
df["month"]=pd.to_datetime(df["date"]).dt.month
df["year"]=pd.to_datetime(df["date"]).dt.year

#Creating dictionary for month, and replacing it in the data.
month={1:"january",2:"February",3:"March",4:"April",5:"May",6:"June",7:"July",8:"August",9:"Septemper",10:"October",
       11:"Novamber",12:"December"}

df["month"]=df["month"].map(month)

df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,month,year
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN,February,2020
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN,February,2020
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN,February,2020
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN,February,2020
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN,February,2020


Some simple tests on the data

In [33]:
#New cases in UK in August 2020
df.loc[df.year == 2020].loc[df.location == 'United Kingdom'].loc[df.month =='August'].new_cases.sum()
#df_subset.loc[df_subset.year == 2020].loc[df_subset.location == 'United Kingdom'].loc[df_subset.month =='August'].new_cases.sum()

32730.0

In [10]:
#SOME SIMPLE TESTS ON THE DATASET
#New cases in UK in August 2020
df.loc[df.year == 2020].loc[df.location == 'United Kingdom'].loc[df.month =='August'].new_cases.sum()
#df_subset.loc[df_subset.year == 2020].loc[df_subset.location == 'United Kingdom'].loc[df_subset.month =='August'].new_cases.sum()

32730.0

In [71]:
df_subset=df[['year','month','iso_code', 'continent','location','total_cases','new_cases', 'total_deaths', 'new_deaths', 'people_vaccinated','people_fully_vaccinated','population']]
df_subset.head(50)

df_subset['location'].unique()
#df_subset[['location', 'new_cases']].groupby(["location"]).head()

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Asia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao',
       'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethi

In [49]:
df_subset.isnull().sum()

year                            0
month                           0
iso_code                        0
continent                    9189
location                        0
total_cases                  2625
new_cases                    2662
total_deaths                19829
new_deaths                  19641
people_vaccinated          114952
people_fully_vaccinated    117831
population                   1016
dtype: int64

In [72]:
#New cases in UK in August 2020
df_subset.loc[df_subset.year == 2020].loc[df_subset.location == 'United Kingdom'].loc[df_subset.month =='August'].new_cases.sum()
#df_subset['year'==2020].head()

32730.0

In [42]:
df_group=df_subset.groupby(["year","month","continent","location","iso_code"]).sum()
df_group.shape
df_group.head()

total_cases                0
new_cases                  0
total_deaths               0
new_deaths                 0
people_vaccinated          0
people_fully_vaccinated    0
population                 0
dtype: int64

In [43]:
#total confirmed cases in 2020
df_group['year'==2020].total_cases.sum()

KeyError: False